# Cryptocurrency load Data

In [1]:
import os
import numpy as np
import pandas as pd
import pickle
import quandl
from datetime import datetime, timedelta, date
import plotly.offline as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
import matplotlib.pyplot as plt

py.init_notebook_mode(connected=True)

In [2]:
#Parameter:
import os
from pathlib import Path

filename = 'BCHARTS-KRAKENUSD.pkl'
load_date = datetime.fromtimestamp(Path(filename).stat().st_mtime)
if load_date.date() == date.today():
    load_from_cache = True
else:
    load_from_cache = False
    
print(load_from_cache)

#load_from_cache = True

True


# Get Bitcoin Data

In [3]:
def get_quandl_data(quandl_id, cache = True):
    '''Download and cache Quandl dataseries'''
    cache_path = '{}.pkl'.format(quandl_id).replace('/','-')
    if cache == True:
        try:
            f = open(cache_path, 'rb')
            df = pickle.load(f)   
            print('Loaded {} from cache'.format(quandl_id))
        except (OSError, IOError) as e:
            print('Downloading {} from Quandl'.format(quandl_id))
            df = quandl.get(quandl_id, returns="pandas")
            df.to_pickle(cache_path)
            print('Cached {} at {}'.format(quandl_id, cache_path))
    else:
        print('Downloading {} from Quandl'.format(quandl_id))
        df = quandl.get(quandl_id, returns="pandas")
        df.to_pickle(cache_path)
        print('Cached {} at {}'.format(quandl_id, cache_path))
    return df

In [4]:
# Pull Kraken BTC price exchange data
btc_usd_price_kraken = get_quandl_data('BCHARTS/KRAKENUSD', cache = load_from_cache)
btc_usd_price_kraken.tail()

Loaded BCHARTS/KRAKENUSD from cache


,Open,High,Low,Close,Volume (BTC),Volume (Currency),Weighted Price
Date,,,,,,,
2019-11-10,8812.1,9147.6,8751.7,9039.1,2916.522947,2.608645e+07,8944.368101
2019-11-11,9039.0,9068.2,8610.0,8717.0,3855.376968,3.378199e+07,8762.305075
2019-11-12,8717.0,8868.1,8560.0,8812.8,3351.960322,2.923287e+07,8721.127760
2019-11-13,8812.8,8839.7,8709.6,8762.1,1944.958847,1.702952e+07,8755.724532
2019-11-14,8763.9,8769.2,8763.7,8769.2,2.490920,2.184298e+04,8769.042685


In [5]:
# Chart the BTC pricing data
btc_trace = go.Scatter(x=btc_usd_price_kraken.index, y=btc_usd_price_kraken['Weighted Price'])
py.iplot([btc_trace])

In [6]:
# Pull pricing data for 3 more BTC exchanges
exchanges = [#'COINBASE',
             'BITSTAMP'#,
             #'ITBIT'
]

exchange_data = {}

exchange_data['KRAKEN'] = btc_usd_price_kraken

for exchange in exchanges:
    exchange_code = 'BCHARTS/{}USD'.format(exchange)
    btc_exchange_df = get_quandl_data(exchange_code, cache = load_from_cache)
    exchange_data[exchange] = btc_exchange_df

Loaded BCHARTS/BITSTAMPUSD from cache


In [7]:
def merge_dfs_on_column(dataframes, labels, col):
    '''Merge a single column of each dataframe into a new combined dataframe'''
    series_dict = {}
    for index in range(len(dataframes)):
        series_dict[labels[index]] = dataframes[index][col]
        
    return pd.DataFrame(series_dict)

# Merge the BTC price dataseries' into a single dataframe
btc_usd_datasets = merge_dfs_on_column(list(exchange_data.values()), list(exchange_data.keys()), 'Weighted Price')
btc_usd_datasets.tail()

,KRAKEN,BITSTAMP
Date,,
2019-11-10,8944.368101,8974.223982
2019-11-11,8762.305075,8746.107058
2019-11-12,8721.127760,8719.639655
2019-11-13,8755.724532,8750.488566
2019-11-14,8769.042685,8767.936450


## Visualize

In [8]:
def df_scatter(df, title, seperate_y_axis=False, y_axis_label='', scale='linear', initial_hide=False):
    '''Generate a scatter plot of the entire dataframe'''
    label_arr = list(df)
    series_arr = list(map(lambda col: df[col], label_arr))
    
    layout = go.Layout(
        title=title,
        legend=dict(orientation="h"),
        xaxis=dict(type='date'),
        yaxis=dict(
            title=y_axis_label,
            showticklabels= not seperate_y_axis,
            type=scale
        )
    )
    
    y_axis_config = dict(
        overlaying='y',
        showticklabels=False,
        type=scale )
    
    visibility = True
    if initial_hide:
        visibility = 'legendonly'
        
    # Form Trace For Each Series
    trace_arr = []
    for index, series in enumerate(series_arr):
        trace = go.Scatter(
            x=series.index, 
            y=series, 
            name=label_arr[index],
            visible=visibility
        )
        
        # Add seperate axis for the series
        if seperate_y_axis:
            trace['yaxis'] = 'y{}'.format(index + 1)
            layout['yaxis{}'.format(index + 1)] = y_axis_config    
        trace_arr.append(trace)

    fig = go.Figure(data=trace_arr, layout=layout)
    py.iplot(fig)
    
# Plot all of the BTC exchange prices
df_scatter(btc_usd_datasets, 'Bitcoin Price (USD) By Exchange')

## Clean Data

In [9]:
# Remove "0" values
btc_usd_datasets.replace(0, np.nan, inplace=True)
# Plot the revised dataframe
df_scatter(btc_usd_datasets, 'Bitcoin Price (USD) By Exchange')

In [10]:
# Calculate the average BTC price as a new column
btc_usd_datasets['avg_btc_price_usd'] = btc_usd_datasets.mean(axis=1)
# Plot the average BTC price
btc_trace = go.Scatter(x=btc_usd_datasets.index, y=btc_usd_datasets['avg_btc_price_usd'])
py.iplot([btc_trace])

# Get Altcoins Data

In [11]:
def get_json_data(json_url, cache_path):
    '''Download and cache JSON data, return as a dataframe.'''
    try:        
        f = open(cache_path, 'rb')
        df = pickle.load(f)   
        print('Loaded {} from cache'.format(json_url))
    except (OSError, IOError) as e:
        print('Downloading {}'.format(json_url))
        df = pd.read_json(json_url)
        df.to_pickle(cache_path)
        print('Cached {} at {}'.format(json_url, cache_path))
    return df

base_polo_url = 'https://poloniex.com/public?command=returnChartData&currencyPair={}&start={}&end={}&period={}'
start_date = datetime.strptime('2015-01-01', '%Y-%m-%d') # get data from the start of 2015
end_date = datetime.now() # up until today
pediod = 86400 # pull daily data (86,400 seconds per day)

def get_crypto_data(poloniex_pair):
    '''Retrieve cryptocurrency data from poloniex'''
    json_url = base_polo_url.format(poloniex_pair, start_date.timestamp(), end_date.timestamp(), pediod)
    data_df = get_json_data(json_url, poloniex_pair)
    data_df = data_df.set_index('date')
    return data_df

In [12]:
altcoins = ['ETH','LTC','XRP','ETC','STR','DASH','SC','XMR','XEM']

altcoin_data = {}
for altcoin in altcoins:
    coinpair = 'BTC_{}'.format(altcoin)
    crypto_price_df = get_crypto_data(coinpair)
    altcoin_data[altcoin] = crypto_price_df

Loaded https://poloniex.com/public?command=returnChartData&currencyPair=BTC_ETH&start=1420066800.0&end=1573732112.317089&period=86400 from cache
Loaded https://poloniex.com/public?command=returnChartData&currencyPair=BTC_LTC&start=1420066800.0&end=1573732112.317089&period=86400 from cache
Loaded https://poloniex.com/public?command=returnChartData&currencyPair=BTC_XRP&start=1420066800.0&end=1573732112.317089&period=86400 from cache
Loaded https://poloniex.com/public?command=returnChartData&currencyPair=BTC_ETC&start=1420066800.0&end=1573732112.317089&period=86400 from cache
Loaded https://poloniex.com/public?command=returnChartData&currencyPair=BTC_STR&start=1420066800.0&end=1573732112.317089&period=86400 from cache
Loaded https://poloniex.com/public?command=returnChartData&currencyPair=BTC_DASH&start=1420066800.0&end=1573732112.317089&period=86400 from cache
Loaded https://poloniex.com/public?command=returnChartData&currencyPair=BTC_SC&start=1420066800.0&end=1573732112.317089&period=86

In [13]:
altcoin_data['ETH'].tail()

,high,low,open,close,volume,quoteVolume,weightedAverage
date,,,,,,,
2019-10-24,0.021840,0.021375,0.021754,0.021624,263.827324,12172.809162,0.021673
2019-10-25,0.021945,0.020763,0.021630,0.020965,323.595122,15123.501633,0.021397
2019-10-26,0.021038,0.018868,0.020950,0.019446,683.430994,35058.129883,0.019494
2019-10-27,0.019602,0.019106,0.019464,0.019295,205.364853,10609.619694,0.019356
2019-10-28,0.019460,0.018977,0.019274,0.019399,80.707050,4194.053180,0.019243


## Convert Prices to USD

In [14]:
# Calculate USD Price as a new column in each altcoin dataframe
for altcoin in altcoin_data.keys():
    altcoin_data[altcoin]['price_usd'] =  altcoin_data[altcoin]['weightedAverage'] * btc_usd_datasets['avg_btc_price_usd']
    
# Merge USD price of each altcoin into single dataframe 
combined_df = merge_dfs_on_column(list(altcoin_data.values()), list(altcoin_data.keys()), 'price_usd')

# Add BTC price to the dataframe
combined_df['BTC'] = btc_usd_datasets['avg_btc_price_usd']

# Chart all of the altocoin prices
df_scatter(combined_df, 'Cryptocurrency Prices (USD)', seperate_y_axis=False, y_axis_label='Coin Value (USD)', scale='log')

# Gold, Silver and Yuan

In [15]:
gold_usd_price_kraken = get_quandl_data('LBMA/GOLD', cache = load_from_cache)
silver_usd_price_kraken = get_quandl_data('LBMA/SILVER', cache = load_from_cache)
yuan_usd_price_kraken = get_quandl_data('BUNDESBANK/BBEX3_D_CNY_USD_CA_AC_000', cache = load_from_cache)


#print('\n Gold: \n',gold_usd_price_kraken.tail(),'\n', gold_usd_price_kraken.index)
#print('\n Silver: \n',silver_usd_price_kraken.tail(),'\n', silver_usd_price_kraken.index)
#print('\n Yuan Renminbi: \n',yuan_usd_price_kraken.tail(),'\n', yuan_usd_price_kraken.index)

Loaded LBMA/GOLD from cache
Loaded LBMA/SILVER from cache
Loaded BUNDESBANK/BBEX3_D_CNY_USD_CA_AC_000 from cache


#  Join Data

In [16]:
data_gold = pd.DataFrame(data = {#'Date_col' : gold_usd_price_kraken.loc['1994-04-01':'2019-11-01'].index,
                                 'Gold' : gold_usd_price_kraken.loc['2015-01-01':'2019-11-01']['USD (AM)']}
                        )
data_silver = pd.DataFrame(data = {#'Date_col': silver_usd_price_kraken.loc['1994-04-01':'2019-11-01'].index,
                                   'Silver': silver_usd_price_kraken.loc['2015-01-01':'2019-11-01']['USD']}
                          )
data_yuan = pd.DataFrame(data = {#'Date_col': yuan_usd_price_kraken.loc['1994-04-01':'2019-11-01'].index,
                                 'Yuan': yuan_usd_price_kraken.loc['2015-01-01':'2019-11-01']['Value']}
                        )
data_ex = data_gold.join(data_silver, how = 'outer', on = 'Date').join(data_yuan, how = 'outer', on = 'Date',sort = True)

data_full = data_ex.join(combined_df, how = 'inner', on = 'Date').dropna()

#data_full = data_full.set_index('Date')
print(data_full)

               Gold  Silver    Yuan         ETH        LTC       XRP  \
Date                                                                   
2016-07-25  1315.00  19.410  6.6860   13.356479   4.032983  0.006194   
2016-07-26  1321.25  19.680  6.6778   12.468899   3.956227  0.005936   
2016-07-27  1320.80  19.580  6.6671   12.727733   3.952143  0.005930   
2016-07-28  1341.30  20.410  6.6597   12.830996   3.958792  0.005895   
2016-07-29  1332.50  20.040  6.6511   12.681646   4.056811  0.006079   
2016-08-01  1348.85  20.510  6.6277   11.410904   3.896913  0.005802   
2016-08-02  1358.15  20.710  6.6451    8.973210   3.541217  0.005422   
2016-08-03  1364.40  20.590  6.6195    9.488270   3.624974  0.005515   
2016-08-04  1351.15  20.160  6.6444   10.849604   3.751704  0.005762   
2016-08-05  1362.60  20.220  6.6406   10.911223   3.719722  0.005895   
2016-08-08  1330.00  19.660  6.6615   11.061051   3.764318  0.006026   
2016-08-09  1332.90  19.700  6.6594   11.836636   3.706767  0.00

In [17]:
# fill missing Data
np.mean([4,5])

4.5

In [21]:
import pandas as pd

import math

idx = pd.date_range(np.min(data_full.index), np.max(data_full.index))
data_full = data_full.reindex(idx,fill_value = None)
print(data_full[:10])

def fill_nans(data):
    ts_data = data[:]
    data_cols = ts_data.columns
    for col in data_cols:
        ts_series = ts_data[col]
        for i in range(len(ts_series)):
            if math.isnan(ts_series[i]):
                foundb4 = False
                j = 0
                foundafter = False
                k = 0
                while foundb4 == False:
                    j += 1
                    if not math.isnan(ts_series[i-j]):
                        priceb4 = ts_series[i-j]
                        foundb4 = True
                        #print(j)
                while foundafter == False:
                    k += 1
                    if not math.isnan(ts_series[i+k]):
                        priceafter = ts_series[i+k]
                        foundafter = True
                        #print(k)
                    
                ts_series[i] = np.mean([priceb4 ,priceafter])
            #ts_series = [i if not math.isnan(i) else 0 for i in ts_series]
        
        
        
        ts_data[col] = ts_series
    
    return(ts_data)

data_full = fill_nans(data_full)
data_full.index.name = 'Date'
print(data_full[:10])


                 Gold   Silver     Yuan        ETH       LTC       XRP  \
2016-07-25  1315.0000  19.4100  6.68600  13.356479  4.032983  0.006194   
2016-07-26  1321.2500  19.6800  6.67780  12.468899  3.956227  0.005936   
2016-07-27  1320.8000  19.5800  6.66710  12.727733  3.952143  0.005930   
2016-07-28  1341.3000  20.4100  6.65970  12.830996  3.958792  0.005895   
2016-07-29  1332.5000  20.0400  6.65110  12.681646  4.056811  0.006079   
2016-07-30  1340.6750  20.2750  6.63940  12.046275  3.976862  0.005940   
2016-07-31  1344.7625  20.3925  6.63355  11.728590  3.936887  0.005871   
2016-08-01  1348.8500  20.5100  6.62770  11.410904  3.896913  0.005802   
2016-08-02  1358.1500  20.7100  6.64510   8.973210  3.541217  0.005422   
2016-08-03  1364.4000  20.5900  6.61950   9.488270  3.624974  0.005515   

                 ETC       STR      DASH        SC       XMR       XEM  \
2016-07-25  0.633601  0.002185  9.108967  0.000656  1.842516  0.006824   
2016-07-26  1.814205  0.001974  8.864

# Save Data

In [22]:
save_path = r'data_full.csv'
data_full.to_csv(save_path)